# Tests

In [1]:
import os
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from termcolor import colored

# change this property
NOMEROFF_NET_DIR = "/var/www/nomeroff-net/"
MASK_RCNN_DIR = "/var/www/Mask_RCNN/"

MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, "logs/")
MASK_RCNN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/mask_rcnn_numberplate_0700.h5")
REGION_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/npregion_2018-12-27-295x64-2.h5")
FILL_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/npfilled_model_2019_1_15.h5")

sys.path.append(NOMEROFF_NET_DIR)

In [2]:
from NomeroffNet import  filters, RectDetector, TextDetector, RegionDetector, Detector, textPostprocessing, FillDetector

nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)

rectDetector = RectDetector()
textDetector = TextDetector()
regionDetector = RegionDetector()
regionDetector.load(REGION_MODEL_PATH)
fillDetector = FillDetector()
fillDetector.load(FILL_MODEL_PATH)

Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib64/python3.6/site-packages/keras/engine/saving.py:304: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [8]:
def test(dirName, fname, y, verbose=0):
    img_path = os.path.join(dirName, fname)
    if verbose==1:
        print(colored(f"__________ \t\t {img_path} \t\t __________", "blue"))
    img = mpimg.imread(img_path)
    NP = nnet.detect([img])
    if verbose==1:
        print(colored("MASK_RCNN RESULTS:", "blue"))
        print(NP)
    cv_img_masks = filters.cv_img_mask(NP) 
    nGood = 0
    nBad = 0
    res = []
    for img_mask in cv_img_masks:
        #mpimg.imsave("masks/mask.png", img_mask)
        points = rectDetector.detect(img_mask, fixRectangleAngle=1, outboundWidthOffset=3)
        if verbose==1:
            print(colored("RECT DETECTOR RESULTS:", "blue"))
            print(points)
        zone = rectDetector.get_cv_zones(img, points)
        #print(fillDetector.predict(zone))
        if verbose==1:
            plt.axis("off")
            plt.imshow(zone)
            plt.show()
        regionId = regionDetector.predict(zone)
        regionName = regionDetector.getLabels(regionId)
        if verbose==1:
            print(colored("REGION NAME:", "blue"))
            print(regionName)
        text = textDetector.detect(zone)
        if verbose==1:
            print(colored("SOURCE TEXT:", "blue"))
            print(text)
        text = textPostprocessing(text, regionName)
        if verbose==1:
            print(colored("POSTPROCESSED TEXT:", "blue"))
            print(text)
        res.append(text)
    for yText in y:
        if yText in res:
            print(colored(f"OK: TEXT:{yText} \t RESULTS:{res} \t\t in PATH:{img_path}", 'green'))
            nGood += 1
        else:
            print(colored(f"NOT OK: TEXT:{yText} \t RESULTS:{res} \t\t in PATH:{img_path} ", 'red'))
            nBad += 1
    return nGood, nBad

In [5]:
dirName = "images/"
testData = {
    "0.jpeg": ["AI5255EI"],
    "1.jpeg": ["HH7777CC"],
    "2.jpeg": ["AT1515CK"],
    "3.jpeg": ["BX0578CE"],
    "4.jpeg": ["AC4249CB"],
    "5.jpeg": ["BC3496HC"],
    "6.jpeg": ["BC3496HC"],
    "7.jpeg": ["AO1306CH"],
    "8.jpeg": ["AE1077CO"],
    "9.jpeg": ["AB3391AK"],
    "10.jpeg": ["BE7425CB"],
    "11.jpeg": ["BE7425CB"],
    "12.jpeg": ["AB0608EA"],
    "13.jpeg": ["AB0680EA"],
    "14.jpeg": ["BM1930BM"],
    "15.jpeg": ["AI1382HB"],
    "16.jpeg": ["AB7333BH"],
    "17.jpeg": ["AB7642CT"],
    "18.jpeg": ["AC4921CB"],
    "19.jpeg": ["BC9911BK"],
    "20.jpeg": ["BC7007AK"],
    "21.jpeg": ["AB5649CI"],
    "22.jpeg": ["AX2756EK"],
    "23.jpeg": ["AA7564MX"],
    "24.jpeg": ["AM5696CK"],
    "25.jpeg": ["AM5696CK"],
}

gGood = 0
gBad = 0
i = 0
for fileName in testData.keys():
    print(f"step {i}")
    nGood, nBad = test(dirName, fileName, testData[fileName])
    gGood += nGood
    gBad += nBad
    i += 1
total = gGood + gBad
print(f"TOTAL GOOD: {gGood/total}")
print(f"TOTAL BED: {gBad/total}")

step 0


/usr/local/lib64/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


OK: TEXT:AI5255EI 	 RESULTS:['AI5255EI'] 		 in PATH:images/0.jpeg
step 1


/usr/local/lib64/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


OK: TEXT:HH7777CC 	 RESULTS:['HH7777CC'] 		 in PATH:images/1.jpeg
step 2


...

OK: TEXT:BC9911BK 	 RESULTS:['BC9911BK'] 		 in PATH:images/19.jpeg
step 20


/usr/local/lib64/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


OK: TEXT:BC7007AK 	 RESULTS:['BC7007AK'] 		 in PATH:images/20.jpeg
step 21


/usr/local/lib64/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


KeyboardInterrupt: 